In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from collections import Counter, defaultdict
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class IALaboPro:
    """IA Labo Pro - Combinaison LSTM + CNN pour analyse temporelle avancée"""
    
    def __init__(self, fichier_donnees):
        # Configuration
        self.numero_max = 49
        self.sequence_length = 10
        self.df = self.charger_donnees(fichier_donnees)
        
    def charger_donnees(self, fichier):
        """Charge et prépare les données"""
        try:
            df = pd.read_csv("lotodata.csv", sep=';')
            print(f"✅ {len(df)} lignes chargées")
            
            # Identifier les colonnes de numéros
            num_cols = [f'num{i}' for i in range(5) if f'num{i}' in df.columns]
            if len(num_cols) < 5:
                num_cols = df.select_dtypes(include=[np.number]).columns[:5].tolist()
            
            df = df[num_cols].copy()
            df = df.dropna()
            
            for col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            df = df.dropna()
            df = pd.DataFrame(np.sort(df.values, axis=1), columns=df.columns)
            
            print(f"📊 {len(df)} tirages préparés")
            return df
            
        except Exception as e:
            print(f"❌ Erreur: {e}")
            return None
    
    # ==================== MODULE HYBRIDE LSTM-CNN ====================
    
    class ModeleHybrideLSTMCNN:
        """Module 1: Modèle hybride LSTM + CNN"""
        
        def __init__(self, historique, sequence_length=10):
            self.historique = historique
            self.sequence_length = sequence_length
            self.model = self.construire_modele_hybride()
            if len(historique) > sequence_length * 2:
                self.entrainer_modele()
            
        def preparer_donnees_sequences(self):
            """Prépare les séquences pour l'entraînement"""
            if len(self.historique) < self.sequence_length + 1:
                return None, None
            
            X, y = [], []
            
            for i in range(len(self.historique) - self.sequence_length):
                # Séquence d'entrée
                sequence = self.historique[i:i+self.sequence_length]
                
                # Encodage
                seq_encoded = self.encoder_sequence(sequence)
                X.append(seq_encoded)
                
                # Cible
                target = self.historique[i+self.sequence_length]
                y.append(self.encoder_tirage(target))
            
            return np.array(X), np.array(y)
        
        def encoder_sequence(self, sequence):
            """Encode une séquence"""
            encoded = []
            for tirage in sequence:
                tirage_encoded = np.zeros(49)
                for num in tirage:
                    if 1 <= num <= 49:
                        tirage_encoded[num-1] = 1
                encoded.append(tirage_encoded)
            
            return np.array(encoded)
        
        def encoder_tirage(self, tirage):
            """Encode un tirage cible"""
            encoded = np.zeros(49)
            for num in tirage:
                if 1 <= num <= 49:
                    encoded[num-1] = 1
            return encoded / 5
        
        def construire_modele_hybride(self):
            """Construit le modèle hybride LSTM + CNN"""
            
            # Entrée
            input_seq = layers.Input(shape=(self.sequence_length, 49))
            
            # LSTM
            lstm_out = layers.LSTM(64, return_sequences=True)(input_seq)
            lstm_out = layers.LSTM(32)(lstm_out)
            
            # CNN 1D
            cnn_out = layers.Conv1D(64, 3, activation='relu', padding='same')(input_seq)
            cnn_out = layers.MaxPooling1D(2)(cnn_out)
            cnn_out = layers.Conv1D(128, 3, activation='relu', padding='same')(cnn_out)
            cnn_out = layers.GlobalAveragePooling1D()(cnn_out)
            
            # Fusion
            fusion = layers.Concatenate()([lstm_out, cnn_out])
            
            # Couches denses
            x = layers.Dense(128, activation='relu')(fusion)
            x = layers.Dropout(0.3)(x)
            x = layers.Dense(64, activation='relu')(x)
            x = layers.Dropout(0.2)(x)
            
            # Sortie
            output = layers.Dense(49, activation='softmax')(x)
            
            model = Model(inputs=input_seq, outputs=output)
            
            model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            print("✅ Modèle hybride LSTM+CNN construit")
            return model
        
        def entrainer_modele(self):
            """Entraîne le modèle"""
            X, y = self.preparer_donnees_sequences()
            if X is None or len(X) < 20:
                print("⚠️  Pas assez de données pour l'entraînement")
                return
            
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
            
            history = self.model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=30,
                batch_size=16,
                verbose=0
            )
            
            print(f"✅ Modèle entraîné - Accuracy: {history.history['val_accuracy'][-1]:.3f}")
        
        def analyser(self):
            """Prédiction"""
            if len(self.historique) < self.sequence_length:
                scores = np.zeros(49)
                for tirage in self.historique:
                    for num in tirage:
                        scores[num-1] += 1
                return scores / scores.sum() if scores.sum() > 0 else np.ones(49)/49
            
            derniere_sequence = self.historique[-self.sequence_length:]
            seq_encoded = self.encoder_sequence(derniere_sequence)
            seq_encoded = np.expand_dims(seq_encoded, axis=0)
            
            try:
                predictions = self.model.predict(seq_encoded, verbose=0)[0]
            except:
                predictions = np.ones(49) / 49
            
            return predictions
    
    # ==================== MODULE CNN 2D ====================
    
    class ModeleCNN2D:
        """Module 2: CNN 2D pour analyse spatiale"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def encoder_grille(self, tirage):
            """Encode un tirage en grille 7x7"""
            grille = np.zeros((7, 7))
            for num in tirage:
                if 1 <= num <= 49:
                    ligne = (num - 1) // 7
                    colonne = (num - 1) % 7
                    grille[ligne, colonne] = 1
            return grille
        
        def analyser(self):
            """Analyse avec CNN 2D"""
            scores = np.zeros(49)
            
            # Fréquence
            for tirage in self.historique[-20:]:
                for num in tirage:
                    scores[num-1] += 1
            
            # Biais spatial
            for num in range(1, 50):
                ligne = (num - 1) // 7
                colonne = (num - 1) % 7
                distance_centre = abs(ligne - 3) + abs(colonne - 3)
                scores[num-1] += (6 - distance_centre) * 0.1
            
            return scores / scores.sum() if scores.sum() > 0 else scores
    
    # ==================== MODULE ATTENTION ====================
    
    class ModeleAttention:
        """Module 3: Mécanisme d'attention"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse avec attention"""
            scores = np.zeros(49)
            
            for i in range(len(self.historique) - 1):
                tirage_actuel = self.historique[i]
                tirage_suivant = self.historique[i+1]
                
                for num in tirage_actuel:
                    for num_suivant in tirage_suivant:
                        scores[num_suivant-1] += 0.1
            
            if scores.sum() > 0:
                scores = scores / scores.sum()
            
            return scores
    
    # ==================== MODULES STATISTIQUES ====================
    
    class AnalyseMultiEchelle:
        """Module 4: Analyse multi-échelles"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse à différentes échelles"""
            scores = np.zeros(49)
            
            # Court terme
            if len(self.historique) >= 10:
                fenetre = self.historique[-10:]
                court_terme = self.analyser_fenetre(fenetre)
                scores += court_terme * 0.5
            
            # Moyen terme
            if len(self.historique) >= 30:
                fenetre = self.historique[-30:]
                moyen_terme = self.analyser_fenetre(fenetre)
                scores += moyen_terme * 0.3
            
            # Long terme
            long_terme = self.analyser_fenetre(self.historique)
            scores += long_terme * 0.2
            
            return scores
        
        def analyser_fenetre(self, fenetre):
            """Analyse une fenêtre"""
            scores = np.zeros(49)
            
            if not fenetre:
                return scores
            
            compteur = Counter()
            for tirage in fenetre:
                compteur.update(tirage)
            
            total = sum(compteur.values())
            for num, count in compteur.items():
                scores[num-1] = count / total
            
            return scores
    
    class AnalyseCycles:
        """Module 5: Analyse des cycles"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Détecte les cycles"""
            scores = np.zeros(49)
            
            for num in range(1, 50):
                indices = []
                for i, tirage in enumerate(self.historique):
                    if num in tirage:
                        indices.append(i)
                
                if len(indices) >= 3:
                    intervals = np.diff(indices)
                    if len(intervals) > 1:
                        regularite = 1.0 / (np.std(intervals) + 1)
                        scores[num-1] = regularite
            
            if scores.max() > 0:
                scores = scores / scores.max()
            
            return scores
    
    class AnalyseGeometrique:
        """Module 6: Analyse géométrique"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des patterns"""
            scores = np.zeros(49)
            
            # Patterns communs
            for tirage in self.historique[-50:]:
                for num in tirage:
                    scores[num-1] += 0.02
            
            # Zones favorisées
            zones = [
                range(1, 10),   # Coin supérieur gauche
                range(40, 50),  # Coin inférieur droit
                range(15, 22),  # Centre
            ]
            
            for zone in zones:
                for num in zone:
                    if num <= 49:
                        scores[num-1] += 0.05
            
            return scores / scores.sum() if scores.sum() > 0 else scores
    
    # ==================== MOTEUR PRINCIPAL ====================
    
    def analyser_avec_modeles_avances(self):
        """Exécute tous les modèles"""
        if self.df is None or len(self.df) < 30:
            print("❌ Minimum 30 tirages requis")
            return None
        
        historique = self.df.values.tolist()
        
        print("🧠 Initialisation des modèles...")
        
        # Initialisation des modèles
        modele_hybride = self.ModeleHybrideLSTMCNN(historique)
        modele_cnn2d = self.ModeleCNN2D(historique)
        modele_attention = self.ModeleAttention(historique)
        modele_multiechelle = self.AnalyseMultiEchelle(historique)
        modele_cycles = self.AnalyseCycles(historique)
        modele_geometrique = self.AnalyseGeometrique(historique)
        
        # Exécution
        resultats = {
            'LSTM-CNN Hybride': modele_hybride.analyser(),
            'CNN 2D Spatial': modele_cnn2d.analyser(),
            'Attention': modele_attention.analyser(),
            'Multi-Échelles': modele_multiechelle.analyser(),
            'Cycles': modele_cycles.analyser(),
            'Géométrique': modele_geometrique.analyser()
        }
        
        print("✅ Tous les modèles exécutés")
        return resultats
    
    def aggregation_intelligente(self, resultats_modeles):
        """Agrégation des résultats"""
        predictions = np.array(list(resultats_modeles.values()))
        
        # Poids selon la complexité
        poids = np.array([0.25, 0.20, 0.15, 0.15, 0.15, 0.10])
        
        if len(poids) > len(predictions):
            poids = poids[:len(predictions)]
            poids = poids / poids.sum()
        
        combined = np.average(predictions, axis=0, weights=poids)
        
        return combined
    
    def analyser_combinaison(self, combinaison):
        """Analyse une combinaison - MÉTHODE MANQUANTE AJOUTÉE"""
        return {
            'somme': sum(combinaison),
            'pairs': sum(1 for n in combinaison if n % 2 == 0),
            'dizaines': len(set((n-1)//10 for n in combinaison)),
            'ecart': max(combinaison) - min(combinaison),
            'ecart_moyen': np.mean([combinaison[i+1] - combinaison[i] for i in range(4)]),
            'score': self.evaluer_combinaison(combinaison)
        }
    
    def evaluer_combinaison(self, combinaison):
        """Évalue une combinaison"""
        score = 0
        
        # Équilibre pairs/impairs
        pairs = sum(1 for n in combinaison if n % 2 == 0)
        if 2 <= pairs <= 3:
            score += 3
        
        # Diversité des dizaines
        dizaines = len(set((n-1)//10 for n in combinaison))
        if dizaines >= 3:
            score += 2
        
        # Somme optimale
        somme = sum(combinaison)
        if 120 <= somme <= 160:
            score += 2
        
        # Écart minimum
        differences = [combinaison[i+1] - combinaison[i] for i in range(4)]
        if min(differences) >= 3:
            score += 1
        
        return score
    
    def generer_combinaisons_intelligentes(self, numeros_recommandes, n_combinaisons=5):
        """Génère des combinaisons intelligentes"""
        combinaisons = []
        
        strategies = [
            self.strategie_equilibree,
            self.strategie_diversifiee,
            self.strategie_optimisee
        ]
        
        for strategy in strategies:
            if len(combinaisons) < n_combinaisons:
                combi = strategy(numeros_recommandes)
                if combi and self.valider_combinaison(combi):
                    combinaisons.append(sorted(combi))
        
        # Compléter si nécessaire
        attempts = 0
        while len(combinaisons) < n_combinaisons and attempts < 100:
            combi = random.sample(numeros_recommandes.tolist(), 5)
            if self.valider_combinaison(combi):
                combinaisons.append(sorted(combi))
            attempts += 1
        
        # Éliminer les doublons
        combinaisons_uniques = []
        combinaisons_tuples = set()
        
        for combi in combinaisons:
            combi_tuple = tuple(combi)
            if combi_tuple not in combinaisons_tuples:
                combinaisons_uniques.append(combi)
                combinaisons_tuples.add(combi_tuple)
        
        return combinaisons_uniques[:n_combinaisons]
    
    def strategie_equilibree(self, numeros):
        """Stratégie équilibrée"""
        pairs = [n for n in numeros if n % 2 == 0]
        impairs = [n for n in numeros if n % 2 == 1]
        
        if len(pairs) >= 2 and len(impairs) >= 3:
            selected = random.sample(pairs, 2) + random.sample(impairs, 3)
            return sorted(selected)
        
        return None
    
    def strategie_diversifiee(self, numeros):
        """Stratégie diversifiée"""
        groupes = defaultdict(list)
        for num in numeros:
            dizaine = (num - 1) // 10
            groupes[dizaine].append(num)
        
        if len(groupes) >= 2:
            selected = []
            dizaines = list(groupes.keys())
            random.shuffle(dizaines)
            
            for dizaine in dizaines[:2]:
                if groupes[dizaine]:
                    selected.append(random.choice(groupes[dizaine]))
            
            while len(selected) < 5:
                num = random.choice(numeros)
                if num not in selected:
                    selected.append(num)
            
            return sorted(selected)
        
        return None
    
    def strategie_optimisee(self, numeros):
        """Stratégie optimisée"""
        best_combi = None
        best_score = -1
        
        for _ in range(50):
            combi = random.sample(numeros.tolist(), 5)
            score = self.evaluer_combinaison(combi)
            if score > best_score:
                best_score = score
                best_combi = combi
        
        return best_combi
    
    def valider_combinaison(self, combinaison):
        """Valide une combinaison"""
        if len(set(combinaison)) != 5:
            return False
        
        pairs = sum(1 for n in combinaison if n % 2 == 0)
        if not (1 <= pairs <= 4):
            return False
        
        if len(set((n-1)//10 for n in combinaison)) < 2:
            return False
        
        somme = sum(combinaison)
        if not (100 <= somme <= 200):
            return False
        
        return True
    
    def generer_recommandations_pro(self):
        """Génère des recommandations"""
        print(f"\n{'='*60}")
        print("🧠 IA LABO PRO - RECOMMANDATIONS AVANCÉES")
        print(f"{'='*60}")
        
        # Analyse
        resultats = self.analyser_avec_modeles_avances()
        if resultats is None:
            return
        
        # Agrégation
        print("\n🔗 Agrégation des prédictions...")
        scores_fusion = self.aggregation_intelligente(resultats)
        
        # Classement
        classement = np.argsort(scores_fusion)[::-1] + 1
        
        # Affichage
        self.afficher_resultats(resultats, scores_fusion, classement)
        
        # Génération de combinaisons
        if len(classement) >= 12:
            combinaisons = self.generer_combinaisons_intelligentes(classement[:12])
            self.afficher_combinaisons(combinaisons)
        
        return {
            'scores': scores_fusion,
            'classement': classement,
            'resultats_detaille': resultats
        }
    
    def afficher_resultats(self, resultats, scores_fusion, classement):
        """Affiche les résultats"""
        print(f"\n📊 PRÉDICTIONS PAR MODÈLE:")
        for nom, scores in resultats.items():
            top3 = np.argsort(scores)[-3:][::-1] + 1
            print(f"   {nom:20} → Top 3: {top3}")
        
        print(f"\n🎯 TOP 15 NUMÉROS RECOMMANDÉS:")
        print("   Rang | Numéro | Score  | Type   | Grille")
        print("   " + "-" * 45)
        
        for i, num in enumerate(classement[:15], 1):
            score = scores_fusion[num-1]
            type_num = "Pair" if num % 2 == 0 else "Impair"
            ligne = (num - 1) // 7 + 1
            colonne = (num - 1) % 7 + 1
            grille = f"{ligne},{colonne}"
            
            print(f"   {i:4} | {num:6} | {score:.4f} | {type_num:6} | {grille}")
    
    def afficher_combinaisons(self, combinaisons):
        """Affiche les combinaisons"""
        print(f"\n🎯 {len(combinaisons)} COMBINAISONS SUGGÉRÉES:")
        
        for i, comb in enumerate(combinaisons, 1):
            analyse = self.analyser_combinaison(comb)
            
            print(f"\n   Option {i}: {comb}")
            print(f"      📊 Statistiques:")
            print(f"        • Somme: {analyse['somme']:3d}")
            print(f"        • Pairs/Impairs: {analyse['pairs']}/{5-analyse['pairs']}")
            print(f"        • Dizaines: {analyse['dizaines']}/5")
            print(f"        • Écart: {analyse['ecart']:3d}")
            print(f"        • Score IA: {analyse['score']:.1f}/8")

# ==================== EXÉCUTION ====================

if __name__ == "__main__":
    print("""
    ╔═══════════════════════════════════════════╗
    ║          IA LABO PRO - VERSION FINALE     ║
    ║         LSTM + CNN + Multi-Échelles       ║
    ╚═══════════════════════════════════════════╝
    """)
    
    try:
        print("🚀 Initialisation...")
        ia_pro = IALaboPro('lotodata1.csv')
        
        if ia_pro.df is not None and len(ia_pro.df) >= 30:
            print(f"✅ {len(ia_pro.df)} tirages prêts")
            
            # Lancement
            resultats = ia_pro.generer_recommandations_pro()
            
            if resultats:
                print("\n" + "="*60)
                print("💡 TECHNOLOGIES UTILISÉES:")
                print("""
                • LSTM-CNN Hybride: Combinaison deep learning
                • CNN 2D: Analyse spatiale sur grille
                • Attention: Focus sur patterns importants
                • Multi-Échelles: Analyse temporelle
                • Cycles: Détection de répétitions
                • Géométrie: Patterns visuels
                """)
                
        else:
            print(f"\n❌ Données insuffisantes")
            print(f"   Disponible: {len(ia_pro.df) if ia_pro.df else 0} tirages")
            
    except Exception as e:
        print(f"\n❌ ERREUR: {e}")


    ╔═══════════════════════════════════════════╗
    ║          IA LABO PRO - VERSION FINALE     ║
    ║         LSTM + CNN + Multi-Échelles       ║
    ╚═══════════════════════════════════════════╝
    
🚀 Initialisation...
✅ 951 lignes chargées
📊 951 tirages préparés
✅ 951 tirages prêts

🧠 IA LABO PRO - RECOMMANDATIONS AVANCÉES
🧠 Initialisation des modèles...
✅ Modèle hybride LSTM+CNN construit
✅ Modèle entraîné - Accuracy: 0.026
✅ Tous les modèles exécutés

🔗 Agrégation des prédictions...

📊 PRÉDICTIONS PAR MODÈLE:
   LSTM-CNN Hybride     → Top 3: [26  9 12]
   CNN 2D Spatial       → Top 3: [44 25 23]
   Attention            → Top 3: [31  6  3]
   Multi-Échelles       → Top 3: [44 22 29]
   Cycles               → Top 3: [ 6 31 30]
   Géométrique          → Top 3: [ 7 48 30]

🎯 TOP 15 NUMÉROS RECOMMANDÉS:
   Rang | Numéro | Score  | Type   | Grille
   ---------------------------------------------
      1 |      6 | 0.1719 | Pair   | 1,6
      2 |     26 | 0.1667 | Pair   | 4,5
     